# TP 4
## Big Data 2023 - UdeSA
### Musich, Cuellar y Servent

In [10]:
import pandas as pd

# Leer la base de datos
df = pd.read_excel('/Users/macbook/Desktop/UdeSa/Materias/Big data/Big-Data/TP3/usu_hogar_T123.xlsx')

# Filtrar observaciones
df_filtrado = df[df['AGLOMERADO'].isin([32, 33])]

In [12]:
dfI = pd.read_excel('/Users/macbook/Desktop/UdeSa/Materias/Big data/Big-Data/Data/usu_individual_T123.xlsx')

dfI = dfI[dfI['AGLOMERADO'].isin([32, 33])]

# Definimos las intersección entre las dos bases como una lista
columnas_duplicadas = set(dfI.columns).intersection(set(df_filtrado.columns))
# Removemos CODUSU y NRO_HOGAR de la lista ya que queremos usarlas para la intersección 
#(y las columnas sobre las que se une no generan duplicados)
columnas_duplicadas.remove("CODUSU")
columnas_duplicadas.remove("NRO_HOGAR")

# Hacemos el merge habiendo dropeado las columnas repetidas
df2 = pd.merge(
    dfI.drop(columnas_duplicadas, axis=1), 
    df_filtrado,
    on= ['NRO_HOGAR', 'CODUSU'])

In [13]:
# Limpieza

def eliminar_categoria_9(df, column_list):
    """
    Elimina la categoría con valor 9 de una serie de variables en un DataFrame.
    
    Parámetros:
    - df: DataFrame donde se encuentran las columnas.
    - column_list: Lista de columnas en las que se quiere eliminar la categoría con valor 9.
    
    Retorna:
    - DataFrame con las categorías con valor 9 eliminadas en las columnas especificadas.
    """
    for column in column_list:
        df = df[df[column] != 9]
    return df


df3 = eliminar_categoria_9(df2, ['NIVEL_ED', 'CH08', 'CH07', 'CAT_OCUP', 'CH15', 'CH16']) 
# Hay más NAs en otras variables
df4 = df3[(df3['CH06'] > 0)] # Edad menor a cero

In [14]:
def columnas_duplicadas(df):
    """
    Esta función verifica si hay columnas con valores idénticos en un DataFrame.
    
    Argumentos:
    - df: DataFrame de pandas
    
    Retorna:
    - Una lista de tuplas con los nombres de las columnas repetidas.
    """
    
    columnas_repetidas = []
    
    for i in range(df.shape[1]):
        for j in range(i+1, df.shape[1]):
            if df.iloc[:, i].equals(df.iloc[:, j]):
                columnas_repetidas.append((df.columns[i], df.columns[j]))
    
    return columnas_repetidas


def eliminar_columnas_duplicadas(df):
    # Obtener las columnas duplicadas
    cols_duplicadas = columnas_duplicadas(df)
    
    # Crear una lista de columnas para eliminar
    columnas_para_eliminar = [col[1] for col in cols_duplicadas]
    
    # Eliminar las columnas
    df = df.drop(columns=columnas_para_eliminar)
    
    return df

In [15]:
eliminar_columnas_duplicadas(df4) # verificamos si hubiera columnas duplicadas y las eliminamos

def drop_high_na(df, threshold=0.7):
    """
    Elimina las columnas y las filas del DataFrame que tienen un porcentaje de valores faltantes 
    superior al umbral especificado.

    Parámetros:
    - df: DataFrame de entrada.
    - threshold: umbral de porcentaje de valores faltantes para eliminar una columna o fila (valor predeterminado = 0.7).

    Retorna:
    - DataFrame después de eliminar las columnas y filas con un alto porcentaje de valores faltantes.
    """
    # Calcula el porcentaje de valores faltantes para cada columna
    missing_percentage_col = df.isnull().mean()
    
    # Filtra las columnas que tienen un porcentaje de valores faltantes superior al umbral
    columns_to_drop = missing_percentage_col[missing_percentage_col > threshold].index
    
    # Elimina las columnas seleccionadas del DataFrame
    df_cleaned = df.drop(columns=columns_to_drop)
    
    # Calcula el porcentaje de valores faltantes para cada fila
    missing_percentage_row = df_cleaned.isnull().mean(axis=1)
    
    # Filtra las filas que tienen un porcentaje de valores faltantes superior al umbral
    rows_to_drop = missing_percentage_row[missing_percentage_row > threshold].index
    
    # Elimina las filas seleccionadas del DataFrame
    df_cleaned = df_cleaned.drop(index=rows_to_drop)
    
    return df_cleaned

drop_high_na(df4)

,CODUSU,NRO_HOGAR,COMPONENTE,H15,CH03,CH04,CH05,CH06,CH07,CH08,...,RDECCFR,GDECCFR,ADECCFR,PONDIH,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4
0,TQRMNORUYHLMSMCDEIJAH00719364,1,1,1,1,2,1957-10-17 00:00:00,65,1,1.0,...,5,5.0,6,2540,1,0,2,0,0,0
1,TQRMNORUYHLMSMCDEIJAH00719364,1,2,1,2,1,1952-04-07 00:00:00,70,1,1.0,...,5,5.0,6,2540,1,0,2,0,0,0
2,TQRMNOSRQHJNSOCDEIJAH00802640,1,1,1,1,1,1986-09-26 00:00:00,36,1,4.0,...,1,1.0,1,14082,2,1,98,0,0,0
3,TQRMNOSRQHJNSOCDEIJAH00802640,1,2,1,2,2,1994-05-02 00:00:00,28,1,4.0,...,1,1.0,1,14082,2,1,98,0,0,0
4,TQRMNOSRQHJNSOCDEIJAH00802640,1,3,0,3,2,2014-03-20 00:00:00,9,5,4.0,...,1,1.0,1,14082,2,1,98,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7614,TQRMNOPVVHKLRQCDEIJAH00780546,1,3,0,3,1,2013-08-05 00:00:00,9,5,2.0,...,9,9.0,9,2295,1,2,98,0,0,0
7615,TQRMNOQPTHKKRRCDEIJAH00780327,1,1,1,1,2,1945-06-05 00:00:00,77,3,1.0,...,10,10.0,10,1516,1,0,96,0,0,0
7616,TQRMNOPTXHKORSCDEIJAH00780269,1,1,1,1,2,1977-07-31 00:00:00,45,5,1.0,...,12,12.0,12,0,1,2,3,0,0,0
7617,TQRMNOPTXHKORSCDEIJAH00780269,1,2,1,3,1,2005-06-17 00:00:00,17,5,1.0,...,12,12.0,12,0,1,2,3,0,0,0


In [16]:
# Crear una copia del DataFrame para evitar SettingWithCopyWarning
df4_copy = df4.copy()

# Ajustamos la variable 'Nivel_ed'. 
df4_copy['NIVEL_ED'].replace(7, 0, inplace=True)

# Convertimos las variables categóricas a formato de cadena
category_list = [ 'CH08', 'CH09', 'T_VI', 'V10_M', 'IV1', 'IV3', 'IV4', 'IV6', 'IV7', 'IV9', 'IV10', 'II7', 'II8', 'II9', 'IV11']
df_categories = df4_copy[category_list].astype(str)

# Creamos variables dummy para las variables categóricas
dummy_data = pd.get_dummies(df_categories, prefix=category_list)

# Fusionamos las variables dummy con el DataFrame original, excluyendo las originales
df4_copy = pd.concat([df4_copy, dummy_data], axis=1)
df4_copy.drop(category_list, axis=1, inplace=True)

In [17]:
df4 = df4_copy.copy()

# Eliminamos las columnas ANO4 y TRIMESTRE de df4
df4.drop(['ANO4', 'TRIMESTRE'], axis=1, inplace=True)

def resumen_dataframe(df):
    print(f"Total de filas: {df.shape[0]}")
    print(f"Total de columnas: {df.shape[1]}")

resumen_dataframe(df4)

Total de filas: 7536
Total de columnas: 572


In [18]:
adultos_data = pd.read_excel("/Users/macbook/Desktop/UdeSa/Materias/Big data/Big-Data/TP2/tabla_adulto_equiv.xlsx", skiprows=3)
adultos_data = adultos_data.dropna(how='all')
adultos_data = adultos_data.iloc[:23, :]

def categorize_age(age):
    if age == 0:
        return "Menor de 1 año"
    elif age == 1:
        return "1 año"
    elif age == 2:
        return "2 años"
    elif age == 3:
        return "3 años"
    elif age == 4:
        return "4 años"
    elif age == 5:
        return "5 años"
    elif age == 6:
        return "6 años"
    elif age == 7:
        return "7 años"
    elif age == 8:
        return "8 años"
    elif age == 9:
        return "9 años"
    elif age == 10:
        return "10 años"
    elif age == 11:
        return "11 años"
    elif age == 12:
        return "12 años"
    elif age == 13:
        return "13 años"
    elif age == 14:
        return "14 años"
    elif age == 15:
        return "15 años"
    elif age == 16:
        return "16 años"
    elif age == 17:
        return "17 años"
    elif 18 <= age <= 29:
        return "18 a 29 años"
    elif 30 <= age <= 45:
        return "30 a 45 años"
    elif 46 <= age <= 60:
        return "46 a 60 años"
    elif 61 <= age <= 75:
        return "61 a 75 años"
    else:
        return "más de 75 años"

# Creando la columna "Rango de Edad"
df4.loc[:, 'Edad'] = df4['CH06'].apply(categorize_age)

# Creando dos dataframes: uno para Mujeres y otro para Varones
df_mujeres = adultos_data[['Edad', 'Mujeres']].copy()
df_mujeres['CH04'] = 2
df_mujeres.rename(columns={'Mujeres': 'adulto_equiv'}, inplace=True)

df_varones = adultos_data[['Edad', 'Varones']].copy()
df_varones['CH04'] = 1
df_varones.rename(columns={'Varones': 'adulto_equiv'}, inplace=True)

# Concatenando ambos dataframes
adultos_equiv_data = pd.concat([df_mujeres, df_varones], axis=0)

# Reordenando las columnas y ordenando el dataframe por Edad
adultos_equiv_data = adultos_equiv_data[['Edad', 'CH04', 'adulto_equiv']]
adultos_equiv_data = adultos_equiv_data.sort_values(by=['Edad', 'CH04']).reset_index(drop=True)

In [19]:
# Uniendo df4 con adultos_equiv_data usando las columnas "Edad" y "CH04"
df5 = pd.merge(df4, adultos_equiv_data[['Edad', 'CH04', 'adulto_equiv']],
               on=['Edad', 'CH04'], how='left')

# Mostrando las primeras filas de la base resultante
df5[['CH04', 'CH06', 'Edad', 'adulto_equiv']].head()

# Agrupar por NRO_HOGAR y CODUSU y calcular la suma de adultos_equiv para cada combinación
sum_adulto_equiv = df5.groupby(['NRO_HOGAR', 'CODUSU'])['adulto_equiv'].sum().reset_index(name='ad_equiv_hogar')

# Fusionar con df5 para asignar los valores calculados a la columna ad_equiv_hogar
df6 = pd.merge(df5, sum_adulto_equiv, on=['NRO_HOGAR', 'CODUSU'], how='left')
df6 = df6.copy()

# Mostrar las primeras 10 filas del DataFrame para verificar
df6[['NRO_HOGAR', 'CODUSU', 'adulto_equiv', 'ad_equiv_hogar']].head(10)

,NRO_HOGAR,CODUSU,adulto_equiv,ad_equiv_hogar
0,1,TQRMNORUYHLMSMCDEIJAH00719364,0.67,1.50
1,1,TQRMNORUYHLMSMCDEIJAH00719364,0.83,1.50
2,1,TQRMNOSRQHJNSOCDEIJAH00802640,1.00,2.45
3,1,TQRMNOSRQHJNSOCDEIJAH00802640,0.76,2.45
4,1,TQRMNOSRQHJNSOCDEIJAH00802640,0.69,2.45
5,1,TQRMNOSWQHJLQRCDEIJAH00796254,1.00,2.00
6,1,TQRMNOSWQHJLQRCDEIJAH00796254,1.00,2.00
7,1,TQRMNOPYVHLMRLCDEIJAH00719346,1.00,2.78
8,1,TQRMNOPYVHLMRLCDEIJAH00719346,0.76,2.78
9,1,TQRMNOPYVHLMRLCDEIJAH00719346,1.02,2.78


In [28]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler

# Identificar columnas binarias
columnas_binarias = [col for col in df6.columns if df6[col].nunique() == 2]

# Identificar columnas continuas, excluyendo las binarias
columnas_numericas = [col for col in df6.select_dtypes(include=['int64', 'float64']).columns if col not in columnas_binarias]

# Crear imputadores
imputador_continuo = SimpleImputer(strategy='mean')
imputador_binario = SimpleImputer(strategy='median')

# Crear escalador robusto para manejar outliers en variables continuas
escalador_robusto = RobustScaler()

# Crear transformadores de columna
transformadores = [
    ('imputar_y_escalar_continuas', Pipeline([('imputador', imputador_continuo), ('escalador', escalador_robusto)]), columnas_numericas),
    ('imputar_binarias', imputador_binario, columnas_binarias)
]

# Crear y aplicar el transformador de columna
preprocesador = ColumnTransformer(transformers=transformadores)
df6_transformado = preprocesador.fit_transform(df6)

# Convertir el resultado a DataFrame
columnas_totales = columnas_numericas + columnas_binarias
df7 = pd.DataFrame(df6_transformado, columns=columnas_totales)

In [29]:
# División entre quienes reportaron ingresos y entre quienes no lo hicieron
# Ingreso neesario según canasta básica

norespondieron = df7[(df7['DECIFR'] == 12) | (df7['DECIFR'] == 13) & (df7['ITF'] == 0)].copy()
respondieron = df7[(df7['DECIFR'] != 12) & (df7['DECIFR'] != 13) & (df7['ITF'] != 0)].copy()

canasta_basica_total = 57371.05

respondieron['ingreso_necesario'] = respondieron['ad_equiv_hogar'] * canasta_basica_total
norespondieron['ingreso_necesario'] = norespondieron['ad_equiv_hogar'] * canasta_basica_total

#9) 
# Variable de conteo casos divididos por la línea de pobreza

respondieron['pobre'] = (respondieron['ITF'] < respondieron['ingreso_necesario']).astype(int)